In [1]:
from docxtpl import * 
from typing import Dict


from models.econometrics import Cost, Currency

import math
from typing import Callable
from models.components import Component
from models.econometrics import Cost
from models.geometry import Orientation
from models.photovoltaic import Photovoltaic

# Site references

In [2]:
#currency values


#reference 1 is USD
Cost.set_exchange(Currency.CLP,908.72)
Cost.set_exchange(Currency.EUR,0.92)

cost_inverter = Cost(1000,Currency.EUR)
cost_inverter.cost_after_tax(Currency.CLP)

# exchange rates https://app.freecurrencyapi.com/dashboard


(987929.1, <Currency.CLP: 'peso chileno'>)

In [3]:
#cspell:disable


#init proyect
from models.components import Tech
from models.geometry import GeoPosition
from models.inventory import Building, Project

psr_bucalemu:Project = Project(
    title='Sistema Fotovoltaico OnGrid-Mixto PSR Bucalemu,\nSanto Domingo',
    building = Building(
        geolocation=GeoPosition(latitude=-33.861702,longitude=-71.657839),
        name='Posta de Salud Rural Bucalemu',
        address='Ruta G-80-I km 30 sn, Localidad Bucalemu,\nSanto Domingo',
        city='Santo Domingo'),
    technology= [Tech.PHOTOVOLTAIC]
    )


# psr_sanenrique:Project = Project(
#     title='Sistema Fotovoltaico OnGrid-Mixto PSR San Enrique, Santo Domingo',
#     building = Building(
#         geolocation=GeoPosition(latitude=-33.9081421,longitude=-71.7159192),
#         name='Posta de Salud Rural de San Enrique',
#         address='Ruta G-866 km 0 sn, Localidad San Enrique, Santo Domingo',
#         city='Santo Domingo'),
#     technology= [Tech.PHOTOVOLTAIC]
#     )



api request URL https://power.larc.nasa.gov/api/temporal/hourly/point?Time=LST&parameters=T2M,ALLSKY_SFC_SW_DNI,ALLSKY_SFC_SW_DIFF,ALLSKY_SRF_ALB,SZA,WS10M&community=RE&latitude=-33.861702&longitude=-71.657839&start=20230101&end=20230102&format=JSON


In [4]:
psr_bucalemu.weather.get_data().head(24)

,date,T2M,ALLSKY_SFC_SW_DNI,ALLSKY_SFC_SW_DIFF,ALLSKY_SRF_ALB,SZA,WS10M,year,month,day,hour
0,2022-12-31 19:00:00,15.99,0.00,0.00,None,None,1.61,2022,12,31,19
1,2022-12-31 20:00:00,15.62,0.00,0.00,None,None,1.69,2022,12,31,20
2,2022-12-31 21:00:00,15.29,0.00,0.00,None,None,1.91,2022,12,31,21
3,2022-12-31 22:00:00,14.89,0.00,0.00,None,None,2.08,2022,12,31,22
4,2022-12-31 23:00:00,14.46,0.00,0.00,None,None,2.37,2022,12,31,23
5,2023-01-01 00:00:00,14.20,0.00,0.00,None,None,2.73,2023,01,01,00
6,2023-01-01 01:00:00,13.96,0.00,0.00,None,None,2.96,2023,01,01,01
7,2023-01-01 02:00:00,13.66,0.00,0.00,None,None,2.99,2023,01,01,02
8,2023-01-01 03:00:00,13.34,0.00,0.00,None,None,3.09,2023,01,01,03
9,2023-01-01 04:00:00,13.16,0.00,0.00,None,None,3.15,2023,01,01,04


# Integration 

## Project components

In [5]:
#cspell: disable

#PV COST MODEL, inscludes price clp per watt, including cost of planel, inverter, instalation and infraestructure.
pv_log_model_on_grid:Callable[[float],float] = lambda size_w: math.floor(-152.6*math.log(size_w)+2605.9) # clp/w
pv_log_model_off_grid:Callable[[float],float] = lambda size_w: math.floor(-414.7*math.log(size_w)+6143.9) # clp/w
#PV linear COST MODEL, inscludes price clp per watt,just panel.
pv_linear:Callable[[float],float] = lambda size_w: 245990/655

#clean previous load
psr_bucalemu.components = {}
from IPython.display import clear_output
clear_output(wait=True)

# assembly projects components
psr_bucalemu.add_component(
    'generación',
    Photovoltaic(psr_bucalemu.weather,'Módulo Primario A',
                reference='https://www.tiendatecnored.cl/modulo-fotovoltaico-650w-canadian-solar.html',
                power=655,
                quantity=5,
                cost=None,
                cost_model=pv_linear,
                orientation=Orientation(15,270-16.46)),
    Photovoltaic(psr_bucalemu.weather,'Módulo Secundario B',
                 reference='https://www.tiendatecnored.cl/modulo-fotovoltaico-650w-canadian-solar.html',
                power=655,
                quantity=5,
                cost=None,
                cost_model=pv_linear,
                orientation=Orientation(15,16.46))
    )
psr_bucalemu.add_component(
    'auxiliar',
    Component('inversor de 7kW',
                model='Voltronic 7kW 48V',
                reference='https://solarbex.com/comprar/inversor-hibrido-7kw-48v-voltronic/',
                specification='Híbrido 7kW Monofásico',
                cost_per_unit=Cost(1255,Currency.EUR)),
    Component('Controlador de Carga',
                model='Enertik 80A 12/24V',
                specification='1000W to 2000 24V output',
                cost_per_unit=Cost(101600,Currency.CLP)),
    Component('medidor bidireccional',
                model='Elster AS1440',
                specification='Trifasico',
                cost_per_unit=Cost(306990,Currency.CLP)),
    Component('almacenamiento',
                model='MP NP12-200',
                specification='Betería 12V 200Ah',
                reference='https://www.tiendatecnored.cl/bateria-gel-ciclo-profundo-12v-200ah.html',
                cost_per_unit=Cost(204889,Currency.CLP),
                quantity=10,
                )
    )
psr_bucalemu.add_component(
    'obras',
    Component(description='instalación eléctrica',
            specification='H/P eléctrico',
            cost_per_unit=Cost(1,currency=Currency.UF),
            quantity=100,
              ),
    Component(description='estructura de soporte',
            specification='elementos fijación',
            cost_per_unit=Cost((4*800+11400/4+4*700+4000+500)),
            quantity=24
            )
)

inside cost pv :  245990.0 Currency.CLP
inside cost pv :  245990.0 Currency.CLP


In [6]:
# psr_bucalemu.components['generación'][1].nominal_power()
#pv_log_model(1500) #clp/w

bucket_list = psr_bucalemu.bucket_list(Currency.CLP)
cost = 0
for item in bucket_list:
    cost += item['amount']
    
#unitary price comparison
print(
    'estimado',cost,
    'on grid',pv_log_model_on_grid(655*10)*655*10,
    'off grid',pv_log_model_off_grid(655*10)*655*10,
    'linear',pv_linear(655*10)*655*10)
bucket_list


estimado 10967899.149999999 on grid 8279200 off grid 16368450 linear 2459900.0


[{'gloss': 'generación',
  'description': 'Módulo Primario A',
  'quantity': 5,
  'amount': 1463640.5,
  'currency': 'peso chileno'},
 {'gloss': 'generación',
  'description': 'Módulo Secundario B',
  'quantity': 5,
  'amount': 1463640.5,
  'currency': 'peso chileno'},
 {'gloss': 'auxiliar',
  'description': 'inversor de 7kW',
  'quantity': 1,
  'amount': 1239851.05,
  'currency': 'peso chileno'},
 {'gloss': 'auxiliar',
  'description': 'Controlador de Carga',
  'quantity': 1,
  'amount': 120904.0,
  'currency': 'peso chileno'},
 {'gloss': 'auxiliar',
  'description': 'medidor bidireccional',
  'quantity': 1,
  'amount': 365318.1,
  'currency': 'peso chileno'},
 {'gloss': 'auxiliar',
  'description': 'almacenamiento',
  'quantity': 10,
  'amount': 2438179.0,
  'currency': 'peso chileno'},
 {'gloss': 'obras',
  'description': 'instalación eléctrica',
  'quantity': 100,
  'amount': 3495090.0,
  'currency': 'peso chileno'},
 {'gloss': 'obras',
  'description': 'estructura de soporte',
  '

In [7]:
psr_bucalemu.get_energy_generation('generación')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   cos_phi              48 non-null     float64
 1   IRR_direct_surface   48 non-null     float64
 2   IRR_diffuse_surface  48 non-null     float64
 3   IRR_ground           48 non-null     float64
 4   IRR_incident         48 non-null     float64
dtypes: float64(5)
memory usage: 2.0 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Temperature_cell  48 non-null     float64
 1   IRR_incident      48 non-null     float64
dtypes: float64(2)
memory usage: 900.0 bytes
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  

UFuncTypeError: ufunc 'add' cannot use operands with types dtype('float64') and dtype('<M8[ns]')